In [48]:
#install.packages(c("tree", "caret", "rpart", "rpart.plot","datasets"))
#install.packages(c( "ggplot2", "datasets","ISLR","randomForest"))
# install.packages(c("lava", "prodlim", "ipred", "ModelMetrics", "foreach", "iterators", "timeDate"), type = "binary")

In [122]:
output_dir <- "C:/Users/user2/Documents/GitHub/Decision_Trees/r/output"


In [49]:
library(tree)
library(caret)
library(rpart)
library(rpart.plot)
library(ggplot2)

## Data Cleaning

### Step 1: Load the Data

The dataset does not have headers, so we need to load it and assign column names.

In [50]:
# Read the data (adjust the path to your data location)
url <- "C:/Users/user2/Documents/GitHub/Decision_Trees/r/input/processed.cleveland.data"
df <- read.csv(url, header = FALSE, na.strings = "?")

head(df)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
2,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
3,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
4,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
5,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
6,56,1,2,120,236,0,0,178,0,0.8,1,0,3,0


### Step 2: Rename Variables

Rename columns in the following order:  
`['age', 'sex', 'cp', 'restbp', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'hd']`

In [51]:
# Rename the columns
colnames(df) <- c('age', 'sex', 'cp', 'restbp', 'chol', 'fbs', 'restecg', 
                  'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'hd')

# Display column names
cat("Column names:\n")
print(colnames(df))

# Display structure
str(df)

Column names:
 [1] "age"     "sex"     "cp"      "restbp"  "chol"    "fbs"     "restecg"
 [8] "thalach" "exang"   "oldpeak" "slope"   "ca"      "thal"    "hd"     
'data.frame':	303 obs. of  14 variables:
 $ age    : num  63 67 67 37 41 56 62 57 63 53 ...
 $ sex    : num  1 1 1 1 0 1 0 0 1 1 ...
 $ cp     : num  1 4 4 3 2 2 4 4 4 4 ...
 $ restbp : num  145 160 120 130 130 120 140 120 130 140 ...
 $ chol   : num  233 286 229 250 204 236 268 354 254 203 ...
 $ fbs    : num  1 0 0 0 0 0 0 0 0 1 ...
 $ restecg: num  2 2 2 0 2 0 2 0 2 2 ...
 $ thalach: num  150 108 129 187 172 178 160 163 147 155 ...
 $ exang  : num  0 1 1 0 0 0 0 1 0 1 ...
 $ oldpeak: num  2.3 1.5 2.6 3.5 1.4 0.8 3.6 0.6 1.4 3.1 ...
 $ slope  : num  3 2 2 3 1 1 3 1 2 3 ...
 $ ca     : num  0 3 2 0 0 0 2 0 1 0 ...
 $ thal   : num  6 3 7 3 3 3 3 3 7 7 ...
 $ hd     : int  0 2 1 0 0 0 3 0 2 1 ...


### Step 3: Remove Missing Values

In [52]:
# Check for missing values
cat("Missing values per column:\n")
print(colSums(is.na(df)))

cat("\nTotal rows before removing NA:", nrow(df), "\n")

# Remove rows with missing values
df <- na.omit(df)

cat("Total rows after removing NA:", nrow(df), "\n")

Missing values per column:
    age     sex      cp  restbp    chol     fbs restecg thalach   exang oldpeak 
      0       0       0       0       0       0       0       0       0       0 
  slope      ca    thal      hd 
      0       4       2       0 

Total rows before removing NA: 303 
Total rows after removing NA: 297 


### Step 4: Create Binary Target Variable `y`

Create a binary variable `y` that equals 1 if the person has heart disease and 0 otherwise.  
In the original dataset, `hd = 0` means no disease, and `hd > 0` means disease.

In [61]:
# Create binary target variable
df$y <- ifelse(df$hd > 0, 1, 0)

# Convert to factor for classification
df$y <- as.factor(df$y)

# Display the distribution
cat("Distribution of target variable y:\n")
print(table(df$y))
cat("\nProportions:\n")
print(prop.table(table(df$y)))

Distribution of target variable y:

  0   1 
160 137 

Proportions:

        0         1 
0.5387205 0.4612795 


### Step 5: Identify and Convert Categorical Variables to Dummy Variables

**Categorical variables in this dataset:**
- `sex`: 1 = male, 0 = female
- `cp`: chest pain type (1, 2, 3, 4)
- `fbs`: fasting blood sugar > 120 mg/dl (1 = true, 0 = false)
- `restecg`: resting electrocardiographic results (0, 1, 2)
- `exang`: exercise induced angina (1 = yes, 0 = no)
- `slope`: slope of peak exercise ST segment (1, 2, 3)
- `ca`: number of major vessels (0-3)
- `thal`: 3 = normal; 6 = fixed defect; 7 = reversable defect

In [62]:
# Convert categorical variables to factors
categorical_vars <- c('sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal')

for (var in categorical_vars) {
    df[[var]] <- as.factor(df[[var]])
}

# Display structure after conversion
str(df)

'data.frame':	297 obs. of  15 variables:
 $ age    : num  63 67 67 37 41 56 62 57 63 53 ...
 $ sex    : Factor w/ 2 levels "0","1": 2 2 2 2 1 2 1 1 2 2 ...
 $ cp     : Factor w/ 4 levels "1","2","3","4": 1 4 4 3 2 2 4 4 4 4 ...
 $ restbp : num  145 160 120 130 130 120 140 120 130 140 ...
 $ chol   : num  233 286 229 250 204 236 268 354 254 203 ...
 $ fbs    : Factor w/ 2 levels "0","1": 2 1 1 1 1 1 1 1 1 2 ...
 $ restecg: Factor w/ 3 levels "0","1","2": 3 3 3 1 3 1 3 1 3 3 ...
 $ thalach: num  150 108 129 187 172 178 160 163 147 155 ...
 $ exang  : Factor w/ 2 levels "0","1": 1 2 2 1 1 1 1 2 1 2 ...
 $ oldpeak: num  2.3 1.5 2.6 3.5 1.4 0.8 3.6 0.6 1.4 3.1 ...
 $ slope  : Factor w/ 3 levels "1","2","3": 3 2 2 3 1 1 3 1 2 3 ...
 $ ca     : Factor w/ 4 levels "0","1","2","3": 1 4 3 1 1 1 3 1 2 1 ...
 $ thal   : Factor w/ 3 levels "3","6","7": 2 1 3 1 1 1 1 1 3 3 ...
 $ hd     : int  0 2 1 0 0 0 3 0 2 1 ...
 $ y      : Factor w/ 2 levels "0","1": 1 2 2 1 1 1 2 1 2 2 ...
 - attr(*, "na.acti

In [63]:
# Create dummy variables using model.matrix

# First, keep the original hd column for reference (we'll remove it later)
# Keep only the variables we need for modeling
vars_to_keep <- c('age', 'sex', 'cp', 'restbp', 'chol', 'fbs', 'restecg', 
                  'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'y')

df_model <- df[, vars_to_keep]

# Create a formula for model.matrix (excluding y)
formula_str <- paste("~ . - y")

# Create dummy variables
X_matrix <- model.matrix(as.formula(formula_str), data = df_model)[, -1]  # Remove intercept

# Combine X and y into a final dataset
df_final <- as.data.frame(X_matrix)
df_final$y <- df_model$y

# Display dimensions
cat("Final dataset dimensions:", dim(df_final), "\n")
cat("Number of features:", ncol(df_final) - 1, "\n")
cat("\nFirst few rows:\n")
head(df_final)

Final dataset dimensions: 297 21 
Number of features: 20 

First few rows:


,age,sex1,cp2,cp3,cp4,restbp,chol,fbs1,restecg1,restecg2,⋯,exang1,oldpeak,slope2,slope3,ca1,ca2,ca3,thal6,thal7,y
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,63,1,0,0,0,145,233,1,0,1,⋯,0,2.3,0,1,0,0,0,1,0,0
2,67,1,0,0,1,160,286,0,0,1,⋯,1,1.5,1,0,0,0,1,0,0,1
3,67,1,0,0,1,120,229,0,0,1,⋯,1,2.6,1,0,0,1,0,0,1,1
4,37,1,0,1,0,130,250,0,0,0,⋯,0,3.5,0,1,0,0,0,0,0,0
5,41,0,1,0,0,130,204,0,0,1,⋯,0,1.4,0,0,0,0,0,0,0,0
6,56,1,1,0,0,120,236,0,0,0,⋯,0,0.8,0,0,0,0,0,0,0,0


---
## 1.2 Data Analysis (8 points)

### 1.2.1 Split Data and Plot Classification Tree (1 point)

In [107]:
#semilla
set.seed(123)

#  training (70%) y testing (30%)
train_index <- sample(seq_len(nrow(df_final)), size = 0.7 * nrow(df_final))
train_data <- df_final[train_index, ]
test_data <- df_final[-train_index, ]

cat("tamaño Training:", nrow(train_data), "\n")
cat("tamaño Testing:", nrow(test_data), "\n")

tamaño Training: 207 
tamaño Testing: 90 


In [108]:
# Train classification tree
tree_model <- rpart(y ~ ., data = train_data, method = "class", 
                    control = rpart.control( cp = 0 ,minsplit = 2,minbucket = 1      )) # rpart.control() maxdepth = 3

# Plot the tree
png(paste0(output_dir, "/tree_before_pruning.png"), width = 1200, height = 800, res = 120)

rpart.plot(tree_model,             
           extra = 101,      
           box.palette = c("pink", "lightblue"),  
           shadow.col = "gray", 
           under = TRUE,         
           faclen = 0,            
           cex = 0.6,             
           main = "Classification Tree - Heart Disease (Before Pruning)")
dev.off()

pdf 
  2

### 1.2.2 Plot Confusion Matrix and Interpret (2 points)

In [109]:
# Make predictions on test set
y_pred <- predict(tree_model, newdata = test_data, type = "class")

# Create confusion matrix
cm <- confusionMatrix(y_pred, test_data$y, positive = "1")

# Display confusion matrix
print(cm)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 31 13
         1 15 31
                                          
               Accuracy : 0.6889          
                 95% CI : (0.5826, 0.7823)
    No Information Rate : 0.5111          
    P-Value [Acc > NIR] : 0.0004711       
                                          
                  Kappa : 0.3781          
                                          
 Mcnemar's Test P-Value : 0.8501067       
                                          
            Sensitivity : 0.7045          
            Specificity : 0.6739          
         Pos Pred Value : 0.6739          
         Neg Pred Value : 0.7045          
             Prevalence : 0.4889          
         Detection Rate : 0.3444          
   Detection Prevalence : 0.5111          
      Balanced Accuracy : 0.6892          
                                          
       'Positive' Class : 1               
                                    

In [ ]:
# graficar confusion matrix
cm_table <- as.data.frame(table(Predicted = y_pred, Actual = test_data$y))

labels <- c("0" = "Does not have HD", "1" = "Has HD")
cm_table$Predicted <- factor(cm_table$Predicted, levels = c("0", "1"), labels = labels)
cm_table$Actual <- factor(cm_table$Actual, levels = c("0", "1"), labels = labels)

p1 <- ggplot(cm_table, aes(x = Actual, y = Predicted, fill = Freq)) +
  geom_tile(color = "white") +
  geom_text(aes(label = Freq), size = 8, color = "white") +
  scale_fill_gradient(low = "lightblue", high = "darkblue") +
  labs(title = "Confusion Matrix - Before Pruning",
       x = "Actual",
       y = "Predicted") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

ggsave(
  filename = paste0(output_dir, "/confusion_matrix_before_pruning.png"),
  plot = p1,  
  width = 8,
  height = 6,
  dpi = 300
)

**Interpretation:**

The confusion matrix shows:
- **True Negatives (TN)**: Correctly predicted no heart disease
- **False Positives (FP)**: Incorrectly predicted heart disease
- **False Negatives (FN)**: Incorrectly predicted no heart disease
- **True Positives (TP)**: Correctly predicted heart disease

The tree likely shows signs of **overfitting** (high training accuracy but lower test accuracy).

### 1.2.3 Fix Overfitting with Cross-Validation (1.5 points)

Generate 50 values of α equally spaced on a logarithmic scale between e⁻¹⁰ and 0.05.  
Use 4-fold cross-validation to select the optimal alpha.

In [128]:
# Generate 50 alpha values on logarithmic scale between e^-10 and 0.05
# Using geometric sequence (logarithmic spacing)
alpha_min <- exp(-10)
alpha_max <- 0.05
alphas <- exp(seq(log(alpha_min), log(alpha_max), length.out = 50))

In [ ]:
# Set up 4-fold cross-validation
set.seed(123)

# Create folds
folds <- createFolds(train_data$y, k = 4, list = TRUE, returnTrain = FALSE)

# Function to calculate accuracy for a given alpha
cv_accuracy <- function(alpha, data, folds) {
  accuracies <- numeric(length(folds))
  
  for (i in seq_along(folds)) {
    # Split into train and validation
    val_idx <- folds[[i]]
    cv_train <- data[-val_idx, ]
    cv_val <- data[val_idx, ]
    
    # Train model with specific alpha (cp parameter)
    model <- rpart(y ~ ., data = cv_train, method = "class",
                   control = rpart.control(cp = alpha, minsplit = 2, minbucket = 1))
    
    # Predict on validation set
    pred <- predict(model, newdata = cv_val, type = "class")
    
    # Calculate accuracy
    accuracies[i] <- mean(pred == cv_val$y)
  }
  
  return(mean(accuracies))
}

# Calculate CV accuracy for each alpha
cv_scores <- sapply(alphas, function(a) cv_accuracy(a, train_data, folds))

cat("Cross-validation completed!\n")

Running 4-fold cross-validation for 50 alpha values...


Cross-validation completed!


In [130]:
# Find optimal alpha
best_idx <- which.max(cv_scores)
best_alpha <- alphas[best_idx]
best_accuracy <- cv_scores[best_idx]

cat("Optimal alpha (cp):", best_alpha, "\n")
cat("Best CV accuracy:", best_accuracy, "\n")

Optimal alpha (cp): 0.02120756 
Best CV accuracy: 0.7872912 


### 1.2.4 Plot Inaccuracy Rate vs Alpha (1.5 points)

In [ ]:
#ratio de inaccuracy
plot_data <- data.frame(alpha = alphas, inaccuracy = inaccuracy_rates)

p2 <-ggplot(plot_data, aes(x = alpha, y = inaccuracy)) +
  geom_line(color = "blue", size = 1) +
  geom_point(color = "blue", size = 2) +
  geom_vline(xintercept = best_alpha, color = "red", linetype = "dashed", size = 1) +
  annotate("text", x = best_alpha * 1.5, y = max(inaccuracy_rates) * 0.9,
           label = paste("Optimal α =", round(best_alpha, 6)),
           color = "red", size = 4) +
  scale_x_log10() +
  labs(title = "Inaccuracy Rate vs Alpha (4-Fold Cross-Validation)",
       x = "Alpha (log scale)",
       y = "Inaccuracy Rate (1 - Accuracy)") +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.5))


 ggsave(
  filename = paste0(output_dir, "/inaccuracy_rate.png"),
  plot = p2,  
  width = 8,
  height = 6,
  dpi = 300
) 


### 1.2.5 Plot Pruned Tree and Confusion Matrix (2 points)

In [ ]:
# tree con alpha óptimo
png(paste0(output_dir, "/tree_after_pruning.png"), width = 1200, height = 800, res = 120)
pruned_tree <- rpart(y ~ ., data = train_data, method = "class",
                     control = rpart.control(cp = best_alpha, minsplit = 2, minbucket = 1))

rpart.plot(pruned_tree,
           type = 4,
           extra = 101,
           under = TRUE,
           faclen = 0,
           cex = 0.8,
           main = paste("Classification Tree - Heart Disease (Pruned, α =", round(best_alpha, 6), ")"))

dev.off()

pdf 
  2

In [ ]:
y_pred_pruned <- predict(pruned_tree, newdata = test_data, type = "class")

# materix confusion
cm_pruned <- confusionMatrix(y_pred_pruned, test_data$y, positive = "1")

print(cm_pruned)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 34 11
         1 12 33
                                          
               Accuracy : 0.7444          
                 95% CI : (0.6416, 0.8306)
    No Information Rate : 0.5111          
    P-Value [Acc > NIR] : 5.043e-06       
                                          
                  Kappa : 0.4889          
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.7500          
            Specificity : 0.7391          
         Pos Pred Value : 0.7333          
         Neg Pred Value : 0.7556          
             Prevalence : 0.4889          
         Detection Rate : 0.3667          
   Detection Prevalence : 0.5000          
      Balanced Accuracy : 0.7446          
                                          
       'Positive' Class : 1               
                                    

In [ ]:
# matrix de confusion podada
cm_table_pruned <- as.data.frame(table(Predicted = y_pred_pruned, Actual = test_data$y))

cm_table_pruned$Predicted <- factor(cm_table_pruned$Predicted, levels = c("0", "1"), 
                                    labels = c("Does not have HD", "Has HD"))
cm_table_pruned$Actual <- factor(cm_table_pruned$Actual, levels = c("0", "1"), 
                                 labels = c("Does not have HD", "Has HD"))

p3 <-ggplot(cm_table_pruned, aes(x = Actual, y = Predicted, fill = Freq)) +
  geom_tile(color = "white") +
  geom_text(aes(label = Freq), size = 8, color = "white") +
  scale_fill_gradient(low = "lightblue", high = "darkblue") +
  labs(title = paste("Confusion Matrix - Pruned Tree (α =", round(best_alpha, 6), ")"),
       x = "Actual",
       y = "Predicted") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        plot.title = element_text(hjust = 0.5))

# guardo la matrix de confusion
ggsave(
  filename = paste0(output_dir, "/confusion_matrix_after_pruning.png"),
  plot = p3,  
  width = 8,
  height = 6,
  dpi = 300
)